In [16]:
import pandas as pd
import requests

# Define the endpoints
endpoints = [
    "https://cloudassistant-dev.cbre.com/ask",
    "https://cloudassistant-qa.cbre.com/ask",
    "https://cloudassistant.cbre.com/ask"
]

# Load the Excel file
input_file = "data-test.xlsx"
output_file = "generated_data.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")

# Ensure the 'user_input' column exists
if 'user_input' not in df.columns:
    raise ValueError("The Excel file must contain a column named 'user_input'.")

# Function to get responses from all endpoints
def get_responses(question):
    payload = {
        "user_id": 4,
        "chat_id": 193,
        "conversation_history": [
            {"role": "user", "content": "hi"},
            {"id": 592, "role": "bot", "content": "Hi there! How can I help you today?", "is_liked": False},
            {"role": "user", "content": question}
        ],
        "question": question
    }

    responses = {}
    for url in endpoints:
        try:
            response = requests.post(url, json=payload, verify=False)
            response.raise_for_status()
            responses[url] = response.json().get("answer", "No answer returned")
        except requests.exceptions.HTTPError as http_err:
            responses[url] = f"HTTP error: {http_err} - Response: {response.text}"
        except Exception as e:
            responses[url] = f"Error: {e}"
    return responses


# Apply the function to each row
df["dev_response"] = ""
df["qa_response"] = ""
df["prod_response"] = ""

for index, row in df.iterrows():
    question = row["user_input"]
    answers = get_responses(question)
    print(answers)
    df.at[index, "dev_response"] = answers[endpoints[0]]
    df.at[index, "qa_response"] = answers[endpoints[1]]
    df.at[index, "prod_response"] = answers[endpoints[2]]

# Save the updated DataFrame to a new Excel file
df.to_excel(output_file, index=False)


{'https://cloudassistant-dev.cbre.com/ask': "Hello! I can certainly help you with setting up email services for your application.\n\n**Cloud Engineering offers Twilio SendGrid as a Service** for sending emails from your application. This platform is hosted within CBRE's Azure 2.0 shared Non-Production and Production subscriptions.\n\n**Key details about SendGrid as a Service:**\n\n*   **Availability:** It is available across all cloud providers and all CBRE-approved regions globally.\n*   **Features:**\n    *   CBRE Enterprise Grade email delivery via `cbre.com`\n    *   SPF authentication\n    *   DKIM signing\n    *   DMARC validation\n\n**To consume this service**, you can request a SendGrid Sub User account.\n\n**To request a SendGrid Sub User account, please use the following ServiceNow form:**\n*   **Sendgrid Sub User Request:** [https://cbre.service-now.com/esc?id=sc_cat_item&sys_id=dd27306a1b4296907e0b6287624bcb2d](https://cbre.service-now.com/esc?id=sc_cat_item&sys_id=dd27306a

In [17]:
df

,user_input,ground_truth,dev_response,qa_response,prod_response
0,I would like to set up email services for my a...,"Yes, Cloud Engineering offers a shared SendGri...",Hello! I can certainly help you with setting u...,**Setting Up Email Services for Your Applicati...,**Yes! CBRE Cloud Engineering offers Twilio Se...
1,"I need access to this TFE project, can you hav...",Please raise ServiceNow ticket to get new TFE ...,## Answer\n\nTo gain access to an existing TFE...,**To gain access to a TFE (Terraform Enterpris...,**How to Get TFE Project Access**\n\nTo reques...
2,What are the supported AWS services?,"EC2, SQL Server (IaaS), RDS, EFS/FSX , S3",## Answer\n\nCBRE supports a variety of AWS se...,**Supported AWS Services at CBRE**\n\nThe foll...,**Supported AWS Services at CBRE**\n\nHere are...
3,Are there any Azure services that will be supp...,Azure PostgreSQL (Will be available by end of ...,"## Answer\n\nYes, several Azure services are p...",**Azure Services Supported at a Later Point**\...,"**Yes, there are several Azure services planne..."
4,What is the BaaS architecture for a Dedicated ...,Transaction log backups run every 15 minutes f...,Here's an overview of the BaaS architecture fo...,**BaaS Architecture for a Dedicated Subscripti...,**BaaS Architecture for Dedicated Subscription...
5,Is there a RACI for this service?,"For BaaS service, the RACI matrix shows clear ...",## Answer\n\nBased on the information retrieve...,**Regarding your question: Is there a RACI for...,"Based on the search results, I could not find ..."
6,What is the SLA for a restoration request?,The SLA for restoration requests varies by app...,While the provided data does not specify a con...,**SLA for a Restoration Request**\n\nThe SLA d...,"Based on the documentation, here are the SLA d..."
7,What are the latest changes that happened to t...,"The last tagging policy was updated on May 15,...",Hello! Here are the latest changes that have h...,**Latest Changes to Tagging**\n\nWe have upcom...,**Important Updates to Azure and AWS Tagging S...
8,What is CloudHealth and tell me more about its...,Cloud Health provides a single solution for mu...,## Answer\n\n**CloudHealth** is a single solut...,**What is CloudHealth?**\n\nCloudHealth is a c...,"We're sorry, but it seems we are having troubl..."


In [22]:
import pandas as pd
import requests
import urllib3

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Define the endpoints
endpoints = [
    # "https://cloudassistant-dev.cbre.com/ask",
    "https://cloudassistant-qa.cbre.com/ask",
    "https://cloudassistant.cbre.com/ask"
]

# Load the Excel file
input_file = "data-test.xlsx"
output_file = "generated_data.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")

# Ensure the 'user_input' column exists
if 'user_input' not in df.columns:
    raise ValueError("The Excel file must contain a column named 'user_input'.")

# Limit to first 2 questions for testing
df = df.head(2)

# Prepare response columns
df["dev_response"] = ""
df["qa_response"] = ""
df["prod_response"] = ""

# Function to get responses from all endpoints
def get_responses(question):
    payload = {
        "user_id": 4,
        "chat_id": 193,
        "conversation_history": [
            {"role": "user", "content": "hi"},
            {"id": 592, "role": "bot", "content": "Hi there! How can I help you today?", "is_liked": False},
            {"role": "user", "content": question}
        ],
        "question": question
    }

    responses = {}
    for url in endpoints:
        try:
            response = requests.post(url, json=payload, verify=False)
            response.raise_for_status()
            responses[url] = response.json().get("answer", "No answer returned")
        except requests.exceptions.HTTPError as http_err:
            responses[url] = f"HTTP error: {http_err} - Response: {response.text}"
        except Exception as e:
            responses[url] = f"Error: {e}"
    return responses

# Apply the function to each row
for index, row in df.iterrows():
    question = row["user_input"]
    answers = get_responses(question)
    df.at[index, "dev_response"] = answers[endpoints[0]]
    df.at[index, "qa_response"] = answers[endpoints[1]]
    # df.at[index, "prod_response"] = answers[endpoints[2]]

# Save the updated DataFrame to a new Excel file
df.to_excel(output_file, index=False)

print("Responses saved to generated_data.xlsx")


Responses saved to generated_data.xlsx


In [ ]:
df

,user_input,ground_truth,dev_response,qa_response,prod_response
0,I would like to set up email services for my a...,"Yes, Cloud Engineering offers a shared SendGri...","Hello! Yes, Cloud Engineering offers a service...",**Setting Up Email Services for Your Applicati...,"**Yes, CBRE offers Twilio SendGrid as an Enter..."
1,"I need access to this TFE project, can you hav...",Please raise ServiceNow ticket to get new TFE ...,"## Answer\n\nTo get access to a TFE project, y...",**Access to TFE Project - How to Request**\n\n...,**How to Get TFE Project Access**\n\nTo get ac...


In [25]:
import pandas as pd
import requests
import urllib3
import time

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Define the endpoints
endpoints = [
    "https://cloudassistant-dev.cbre.com/ask",
    "https://cloudassistant-qa.cbre.com/ask",
    "https://cloudassistant.cbre.com/ask"
]

# Load the Excel file
input_file = "data-test.xlsx"
output_file = "generated_data.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")

# Ensure the 'user_input' column exists
if 'user_input' not in df.columns:
    raise ValueError("The Excel file must contain a column named 'user_input'.")

# Limit to first 2 questions for testing
df = df.head(2)

# Prepare response and timing columns
df["dev_response"] = ""
df["qa_response"] = ""
df["prod_response"] = ""
df["dev_time"] = 0.0
df["qa_time"] = 0.0
df["prod_time"] = 0.0

# Function to get responses and timing
def get_responses_with_time(question):
    payload = {
        "user_id": 4,
        "chat_id": 193,
        "conversation_history": [
            {"role": "user", "content": "hi"},
            {"id": 592, "role": "bot", "content": "Hi there! How can I help you today?", "is_liked": False},
            {"role": "user", "content": question}
        ],
        "question": question
    }

    responses = {}
    times = {}
    for url in endpoints:
        start_time = time.time()
        try:
            response = requests.post(url, json=payload, verify=False)
            response.raise_for_status()
            responses[url] = response.json().get("answer", "No answer returned")
        except requests.exceptions.HTTPError as http_err:
            responses[url] = f"HTTP error: {http_err} - Response: {response.text}"
        except Exception as e:
            responses[url] = f"Error: {e}"
        finally:
            times[url] = round(time.time() - start_time, 3)  # seconds
    return responses, times

# Apply the function to each row
for index, row in df.iterrows():
    question = row["user_input"]
    answers, times = get_responses_with_time(question)
    df.at[index, "dev_response"] = answers.get("https://cloudassistant-dev.cbre.com/ask", "")
    df.at[index, "qa_response"] = answers.get("https://cloudassistant-qa.cbre.com/ask", "")
    df.at[index, "prod_response"] = answers.get("https://cloudassistant.cbre.com/ask", "")
    df.at[index, "dev_time"] = times.get("https://cloudassistant-dev.cbre.com/ask", 0.0)
    df.at[index, "qa_time"] = times.get("https://cloudassistant-qa.cbre.com/ask", 0.0)
    df.at[index, "prod_time"] = times.get("https://cloudassistant.cbre.com/ask", 0.0)

# Save the updated DataFrame to a new Excel file
df.to_excel(output_file, index=False)

print("Responses and timings saved to generated_data.xlsx")


KeyboardInterrupt: 

In [26]:
import pandas as pd
import requests
import urllib3
import time
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# -------------------- Logging Setup --------------------
logging.basicConfig(
    filename='response_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.info("Script started.")

# -------------------- Disable SSL Warnings --------------------
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# -------------------- Define Endpoints --------------------
endpoints = [
    "https://cloudassistant-dev.cbre.com/ask",
    "https://cloudassistant-qa.cbre.com/ask",
    "https://cloudassistant.cbre.com/ask"
]

# -------------------- Load Excel File --------------------
input_file = "data-test.xlsx"
output_file = "generated_data.xlsx"
df = pd.read_excel(input_file, engine="openpyxl")

if 'user_input' not in df.columns:
    logging.error("Missing 'user_input' column in Excel file.")
    raise ValueError("The Excel file must contain a column named 'user_input'.")

df = df.head(2)  # Limit for testing

# -------------------- Prepare Columns --------------------
df["dev_response"] = ""
df["qa_response"] = ""
df["prod_response"] = ""
df["dev_time"] = 0.0
df["qa_time"] = 0.0
df["prod_time"] = 0.0

# -------------------- Parallel Request Function --------------------
def get_responses_with_time_parallel(question):
    payload = {
        "user_id": 4,
        "chat_id": 194,
        "conversation_history": [
            {"role": "user", "content": "hi"},
            {"id": 592, "role": "bot", "content": "Hi there! How can I help you today?", "is_liked": False},
            {"role": "user", "content": question}
        ],
        "question": question
    }

    responses = {}
    times = {}

    def fetch_response(url):
        start_time = time.time()
        try:
            response = requests.post(url, json=payload, verify=False)
            response.raise_for_status()
            result = response.json().get("answer", "No answer returned")
            logging.info(f"Success from {url} | Time: {round(time.time() - start_time, 3)}s")
        except requests.exceptions.HTTPError as http_err:
            result = f"HTTP error: {http_err} - Response: {response.text}"
            logging.error(f"HTTP error from {url}: {http_err}")
        except Exception as e:
            result = f"Error: {e}"
            logging.error(f"Exception from {url}: {e}")
        elapsed = round(time.time() - start_time, 3)
        return url, result, elapsed

    with ThreadPoolExecutor(max_workers=len(endpoints)) as executor:
        future_to_url = {executor.submit(fetch_response, url): url for url in endpoints}
        for future in as_completed(future_to_url):
            url, result, elapsed = future.result()
            responses[url] = result
            times[url] = elapsed

    return responses, times

# -------------------- Process Each Question --------------------
for index, row in df.iterrows():
    question = row["user_input"]
    logging.info(f"Processing question {index + 1}: {question}")
    answers, times = get_responses_with_time_parallel(question)

    df.at[index, "dev_response"] = answers.get("https://cloudassistant-dev.cbre.com/ask", "")
    df.at[index, "qa_response"] = answers.get("https://cloudassistant-qa.cbre.com/ask", "")
    df.at[index, "prod_response"] = answers.get("https://cloudassistant.cbre.com/ask", "")
    df.at[index, "dev_time"] = times.get("https://cloudassistant-dev.cbre.com/ask", 0.0)
    df.at[index, "qa_time"] = times.get("https://cloudassistant-qa.cbre.com/ask", 0.0)
    df.at[index, "prod_time"] = times.get("https://cloudassistant.cbre.com/ask", 0.0)

# -------------------- Save Output --------------------
df.to_excel(output_file, index=False)
logging.info("Responses and timings saved to generated_data.xlsx")
print("Responses and timings saved to generated_data.xlsx")


Responses and timings saved to generated_data.xlsx


In [27]:
df

,user_input,ground_truth,dev_response,qa_response,prod_response,dev_time,qa_time,prod_time
0,I would like to set up email services for my a...,"Yes, Cloud Engineering offers a shared SendGri...","## Answer\n\nYes, if you need to set up email ...","We're sorry, but it seems we are having troubl...",**Yes! CBRE offers Twilio SendGrid as an Enter...,16.159,2.177,22.037
1,"I need access to this TFE project, can you hav...",Please raise ServiceNow ticket to get new TFE ...,## Answer\n\nTo gain access to an existing TFE...,**Accessing a TFE Project at CBRE**\n\nTo gain...,"**To get access to a TFE project, here's what ...",21.872,15.177,24.138
